In [1]:
import shap
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import pyspark
from azure.digitaltwins.core import DigitalTwinsClient
from azure.identity import DefaultAzureCredential
from azure.identity import VisualStudioCodeCredential
import os

In [19]:
!az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "0504f721-d451-402b-b884-381428559e39",
    "id": "2320a8eb-1fce-45ea-9dec-ce93cf71ea5c",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure for Students",
    "state": "Enabled",
    "tenantId": "0504f721-d451-402b-b884-381428559e39",
    "user": {
      "name": "h1548782@s.wu.ac.at",
      "type": "user"
    }
  }
]


In [52]:
#getting the credentials

#define the URL of your Digital Twin instance on the Azure platzform
url = "SeleniumForest.api.weu.digitaltwins.azure.net"

#store the gathered credentials in a variable
credential = DefaultAzureCredential()
#create an instance of the Digital Twin Client
#It can be resued later on
global service_client
service_client = DigitalTwinsClient(url, credential)

In [3]:
#store the CSV content in a dataframe
df = pd.read_csv('./Data/right_arm.csv')
df.head()

#df.shape[0]

,Timestamp,Actual Joint Positions,Actual Joint Velocities,Actual Joint Currents,Actual Cartesian Coordinates,Actual Tool Speed,Generalized Forces,Temperature of Each Joint,Execution Time,Safety Status,Tool Acceleration,Norm of Cartesion Linear Momentum,Robot Current,Joint Voltages,Elbow Position,Elbow Velocity,Tool Current,Tool Temperature,TCP Force,Anomaly State
0,257258.126,"[-1.5707390944110315, -1.5707948964885254, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.18499481678009033, 1.1043120622634888, 1.38...","[-0.133089990415502, -0.4433227297722474, 0.48...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-0.015098106867198664, -0.38619858377727334, ...","[34.375, 36.875, 39.6875, 42.8125, 45.0, 45.3125]",0.833027,1,"[-9.423578262329102, 0.26815059781074524, 0.11...",0.000000,0.560617,"[47.48460006713867, 47.61359786987305, 47.6279...","[-0.07999999993886807, -4.2302395934749395e-06...","[0.0, 0.0, 0.0]",0.069408,39.6250,0.693609,0
1,257258.176,"[-1.5707176367389124, -1.570843359033102, -1.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.16534548997879028, 1.0664139986038208, 1.40...","[-0.13308335724829182, -0.44333549960206997, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-0.04213328588448917, -0.4293152706290817, 0....","[34.375, 36.875, 39.6875, 42.8125, 45.0, 45.3125]",0.898871,1,"[-9.346963882446289, 0.2298433631658554, 0.114...",0.000000,0.497956,"[47.5275993347168, 47.64226150512695, 47.65659...","[-0.07999999890089683, -1.7749906051421094e-05...","[0.0, 0.0, 0.0]",0.064909,39.6250,0.685211,0
2,257258.226,"[-1.570730988179342, -1.5708185635008753, -1.5...","[0.0, 0.0, 0.0, -0.010884314775466919, 0.0, 0.0]","[0.1645040214061737, 1.0770467519760132, 1.354...","[-0.13307894938257267, -0.44333179729023336, 0...","[-2.003684872597315e-06, -0.000926684519370717...","[0.05686369196465915, -0.2101876951251029, 0.9...","[34.375, 36.875, 39.6875, 42.8125, 45.0, 45.3125]",1.048140,1,"[-9.385271072387695, 0.2298433631658554, 0.191...",0.004019,0.504669,"[47.513267517089844, 47.670928955078125, 47.67...","[-0.07999999952532996, -1.0642838973058337e-05...","[0.0, 0.0, 0.0]",0.071274,39.6250,0.944428,0
3,257258.276,"[-1.570660416279928, -1.5707827371409913, -1.5...","[0.0, 0.0, 0.0, -0.08753828704357147, 0.0, 0.0]","[0.18218308687210083, 1.0735350847244263, 1.37...","[-0.13305178855972016, -0.443525353390802, 0.4...","[-1.5362262575268616e-05, -0.00740704821459850...","[0.014812691800918066, -0.0035357919273737524,...","[34.375, 36.875, 39.6875, 42.8125, 45.0, 45.3125]",1.124447,1,"[-9.327810287475586, 0.2298433631658554, 0.191...",0.026350,0.506904,"[47.55626678466797, 47.713924407958984, 47.670...","[-0.07999999976091744, -7.5630809928628735e-06...","[0.0, 0.0, 0.0]",0.071954,39.6250,0.891657,0
4,257258.326,"[-1.570700470601217, -1.570839067498678, -1.57...","[0.0, 0.0, 0.0, -0.14845426380634308, 0.0, 0.0]","[0.1679745316505432, 1.0931836366653442, 1.374...","[-0.13306762017073243, -0.44405501412764015, 0...","[-2.5524599112393154e-05, -0.01234826561117640...","[-0.0066400665555922875, -0.33964563763481687,...","[34.375, 36.875, 39.6875, 42.75, 45.0, 45.3125]",1.072475,1,"[-9.404424667358398, 0.3064578175544739, 0.057...",0.048680,0.457674,"[47.55626678466797, 47.670928955078125, 47.685...","[-0.07999999868460166, -1.8077993842428327e-05...","[0.0, 0.0, 0.0]",0.066182,39.6875,0.750271,0


In [6]:
#replace blank spaces in the column names with '_'
df.columns = df.columns.str.replace(' ', '_')

#use only required columns
df = df[['Norm_of_Cartesion_Linear_Momentum', 'Robot_Current', 'Tool_Current', 'Tool_Temperature', 'TCP_Force', 'Anomaly_State']]

################################
##make it smaller for testing###
#since the dataset is quite big 
#one can keep this code block, to
#shrink the dataset#############
def extract_every_nth_row(df, n):
    new_df = df.iloc[::n].copy()
    return new_df

n = 1000

df = extract_every_nth_row(df, n)
################################
################################

#seperate the features and the target
#X and y 
#features
X = df[['Norm_of_Cartesion_Linear_Momentum', 'Robot_Current', 'Tool_Current', 'Tool_Temperature', 'TCP_Force']]
#target
y = df['Anomaly_State'] 

#Train & Test Split
#for now the testsize is 20% of the total dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#reset the index to make the subsets iterateable again
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [45]:
import time
import anomaly_detection as ad
import digital_twin_azure as dt

def start_machine(model_name: str, df, df_sim, index, service_client):
        print('Machine starting up...')
        print('Training ML Algorithm...')
        rf = ad.train_model(X_train, X_test, y_train, y_test)
        print('Model has been trained')
        count = 0

        while True:
                y_predicted = ad.predict_model(X_test, index, rf)
                dt.update_machine_2('Robo', X_test, index, y_predicted, service_client)
            

In [60]:
import digital_twin_azure as dt
import anomaly_detection as ad
index = 0

dt.update_machine_2('RoboArm', X_test, index, y_predicted, service_client)

TypeError: update_machine_2() takes 4 positional arguments but 5 were given